# OpenAI queries with tools (functions)
### - provide OpenAI API with different functions to call in addition to its own knowledge base
- Wolfram Alpha API: provide OpenAI access to real-time data, mathematical and scientific knowledge
- Talk to your own database with the help of chatgpt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from dotenv import load_dotenv
import os

# load your API key to the environment variables
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
import wolframalpha
import ssl
import requests
import certifi

# workaround for mac to solve SSL: CERTIFICATE_VERIFY_FAILED Error
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# provide OpenAI access to real-time data, mathematical and scientific knowledge
# llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
wolfram = WolframAlphaAPIWrapper()

# provide OpenAI access to data in specific database (in this case: kaggle Amazon 
# database about customers' reviews and their helpfulness)
db = SQLDatabase.from_uri("sqlite:///data/database.sqlite")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [ ]:
tools = [
    Tool(
        name="Calculator",
        #func=llm_math_chain.run,
        func=wolfram.run,
        description="useful for when you need to answer questions about real time data, mathematical computations or science problems"
    ),
    Tool(
        name="kaggle_amazon_db",
        func=db_chain.run,
        description="useful for when you need to answer questions about amazon customers' \
            reviews and their helpfulness. Input should be in the form of a question containing full context"
    )
]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
answer = agent.run("What is the name of the user who wrote the largest number of helpful reviews for amazon?")

In [ ]:
try:
    answer = agent.run("What is 2x+5 = -3x + 7?")
except Exception as e:
    with open('error_out.txt', 'w') as out_file:
        out_file.write(str(e))

In [ ]:
answer = agent.run("How far is it from Chicago to Tokyo?")
print(answer)

In [ ]:
answer = agent.run("Make a picture of an airy function using the Calculator tool.")

## use the wolframalpha library directly to get access to detailed parameters

- the langchain WolframAlphaAPIWrapper does not give access to request parameters or detailed response of the api
- by tuning the request parameters it is  possible to get images from the wolfram alpha API like wit the chatgpt plus wolfram alpha plugin

In [ ]:
client = wolframalpha.Client(os.getenv('WOLFRAM_ALPHA_APPID'))
  
res = client.query('How far is it from Chicago to Tokyo?')
  
answer = next(res.results)
print(answer)

### make direct request without any wolfram alpha library

In [ ]:
function = "sin x cos y"
wa_appid = os.getenv('WOLFRAM_ALPHA_APPID')
query = f"plot {function}"
query_url = f"http://api.wolframalpha.com/v2/query?" \
            f"appid={wa_appid}" \
            f"&input={query}" \
            f"&output=json" \
            f"&includepodid=3DPlot" \
            f"&includepodid=ContourPlot"

r = requests.get(query_url).json()

In [ ]:
import requests
from IPython.display import Image, display

image_url = r["queryresult"]["pods"][0]["subpods"][0]["img"]["src"]
#r2 = requests.get(image_url)

Image(url=image_url,format='gif')

In [ ]:
function = "How far is the distance between Chicago and Tokyo?"
wa_appid = os.getenv('WOLFRAM_ALPHA_APPID')
query = f"plot {function}"
query_url = f"http://api.wolframalpha.com/v2/query?" \
            f"appid={wa_appid}" \
            f"&input={query}" \
            f"&output=json" \
            f"&includepodid=3DPlot" \
            f"&includepodid=ContourPlot"

r = requests.get(query_url).json()

In [ ]:
try:
    #image_url2 = r["queryresult"]["pods"][0]["subpods"][0]["img"]["src"]
    pods = r["queryresult"].get('pods', None)
    if pods:
        image_url2 = pods[0]["subpods"][0]["img"]["src"]
        Image(url=image_url2,format='gif')
except Exception as e:
    print(f"wolfram alpha result json structure does not match common structure: {e}")

## References:
- https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent
- https://python.langchain.com/docs/integrations/tools/wolfram_alpha
- https://api.python.langchain.com/en/latest/utilities/langchain.utilities.wolfram_alpha.WolframAlphaAPIWrapper.html
- https://api.python.langchain.com/en/latest/_modules/langchain/tools/wolfram_alpha/tool.html
- https://towardsdatascience.com/build-your-next-project-with-wolfram-alpha-api-and-python-51c2c361d8b9
- https://www.packtpub.com/article-hub/unleashing-the-power-of-wolfram-alpha-api-with-python-and-chatgpt
- https://blog.finxter.com/openai-api-functions-embeddings-course-4-7-database-querying-using-chatgpt/
- https://medium.com/mlearning-ai/chatting-with-your-database-using-langchain-e27893eb840a
